In [8]:
####################################
## A. Librerias
####################################
import os
import time
import json

import datetime
import pandas as pd

from utils.utils import *
from utils.paths import *

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

####################################
## B. Funciones
####################################

def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def columns_to_string(df):
    columns = list(df.columns)    
    for column in columns:
        df[column] = df[column].astype(str)
    return df

####################################
## C. Configuración de parametros
####################################

HEADLESS = False
PRINT_VIEW = False
# CHROME_DRIVER_PATH = False

In [9]:
########################
## D. Ejecución
########################

# 1. Iniciar el driver, el servidor proxy y el proxy
driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=PRINT_VIEW, headless=HEADLESS)
print('1. Se inició el driver, el servidor proxy y el proxy')

Se inicia el servidor en Windows
Usando el chromedriver_path: C:\GitHubProjets\ScrapTrello\chromedriver\windows\chromedriver.exe
1. Se inició el driver, el servidor proxy y el proxy


In [10]:
# 2. Eliminar todos los archivos de la carpeta
try:
    os.mkdir('jsons')
    print('Se creó la carpeta ./jsons')
except:
    print('Ya existe  ./jsons')

try:
    os.mkdir('jsons/trello')
    print('Se creó la carpeta ./jsons/trello')
except:
    print('Ya existe ./jsons/trello')

try:
    os.mkdir(f'jsons/trello')
    print(f'Se creó la carpeta ./jsons/trello')
except:
    print(f'Ya existe ./jsons/trello')

try:
    files = get_files(os.path.join(os.getcwd(),'jsons','trello'), '.')
    for file in files:
        os.remove(file)
    print('Se eliminaron los archivos de la carpeta')
except:
    pass

Ya existe  ./jsons
Se creó la carpeta ./jsons/trello
Ya existe ./jsons/trello
Se eliminaron los archivos de la carpeta


(204, None)

In [11]:
# 3. Iniciar .har
options = {
    'captureHeaders': True, # Capturar los headers --> Default: False
    'captureContent': True, # Capturar el contenido --> Default: False
    'captureBinaryContent': False, # Capturar el contenido binario --> Default: False
    'captureCookies': True, # Capturar las cookies --> Default: False
    'captureHeadersSize': -1, 
    'captureMaxSize': -1, 
    'captureBinaryContentMaxLength': -1 # Tamaño máximo del contenido binario
}
proxy.new_har("trello", options=options)

# 4. Ingresa a la página
url = 'https://trello.com/b/oEho1Tvx/data-operations-bi'
driver.get(url)

# 5. Obtener el .har y guardar los json --> 42743152
time.sleep(10)
har = proxy.har
har

In [27]:
# 6. Guardar el json
i = 0
for  entrie in har.get('log').get('entries'):
    string_content = entrie.get('response').get('content').get('text')

    if string_content is not None:
        if '[{"_id":' in string_content:
            data_list = json.loads(string_content)
            with open(os.path.join(os.getcwd(),'jsons','trello','trello_data.json'), 'w') as json_file:
                json.dump(data_list, json_file, indent=4)
            
            print(f'Se guardó el json {i}')
    i += 1

Se guardó el json 8


In [28]:
# 7. Leer el json
with open('./jsons/trello/trello_data.json', "r") as archivo:
    data = json.load(archivo)

In [40]:
# 8. Leer listas
df_lists = pd.DataFrame(data['lists'])[['id','name','idBoard']]
df_lists

,id,name,idBoard
0,63eabf9c6fa31f96682eabc1,Backlog,63eabebf701eb9c61b32042c
1,63eabfaa52a1d257951c4f67,Doing,63eabebf701eb9c61b32042c
2,64b547a035622ed264e86069,Done Q3,63eabebf701eb9c61b32042c
3,63eabfc1fe437cc6aebf7d3c,Done Q2,63eabebf701eb9c61b32042c


In [41]:
har

{'log': {'version': '1.2',
  'creator': {'name': 'BrowserMob Proxy', 'version': '2.1.4', 'comment': ''},
  'pages': [{'id': 'trello',
    'startedDateTime': '2023-08-29T16:54:48.130-05:00',
    'title': 'trello',
    'pageTimings': {'comment': ''},
    'comment': ''}],
  'entries': [{'pageref': 'trello',
    'startedDateTime': '2023-08-29T16:54:48.253-05:00',
    'request': {'method': 'GET',
     'url': 'https://trello.com/b/oEho1Tvx/data-operations-bi',
     'httpVersion': 'HTTP/1.1',
     'cookies': [{'name': 'ajs_anonymous_id',
       'value': '%22b81af33c-a477-4514-bb56-fa6c9fc1ca21%22',
       'comment': ''},
      {'name': 'dsc',
       'value': '25de1dd7f6443d2fec3f1a5abb25707ebcc71e17ec1934f44e1b55d14ec3654f',
       'comment': ''},
      {'name': 'preAuthProps',
       'value': 's%3A.2VjpSmRyI7YtEFsqX%2FLhRM5wYJxMDerar%2B5ICyIuspc',
       'comment': ''}],
     'headers': [{'name': 'Host', 'value': 'trello.com'},
      {'name': 'Connection', 'value': 'keep-alive'},
      {'nam

In [31]:
data.keys()

dict_keys(['id', 'name', 'closed', 'dateLastActivity', 'dateLastView', 'datePluginDisable', 'enterpriseOwned', 'nodeId', 'idMemberCreator', 'idOrganization', 'prefs', 'premiumFeatures', 'shortLink', 'shortUrl', 'url', 'creationMethod', 'idEnterprise', 'desc', 'descData', 'idTags', 'labelNames', 'limits', 'powerUps', 'subscribed', 'templateGallery', 'boardPlugins', 'cards', 'enterprise', 'labels', 'lists', 'members', 'customFields', 'memberships', 'myPrefs', 'pluginData'])